In [ ]:
!pip install ortools

In [ ]:
import pandas as pd
from ortools.linear_solver import pywraplp

def calcular_requisitos_nutricionais(genero, peso):
    # Calcula os requisitos nutricionais baseados no gênero e peso
    if genero == 'HOMEM':
        # Para homens, os valores dos requisitos nutricionais são diferentes
        car = 4 * peso  # Carboidratos
        pro = 0.8 * peso  # Proteínas
        gordura = peso  # Gorduras
        vta = 900  # Vitamina A
        vtc = 90  # Vitamina C
        calc = 1000  # Cálcio
        fer = 8  # Ferro
    elif genero == 'MULHER':
        # Para mulheres, os valores dos requisitos nutricionais são diferentes, especialmente se o peso for maior que 50kg
        if peso > 50:
            car = (peso - 50) * 3 + 200
            pro = (peso - 50) * 0.6 + 40
            gordura = peso
            vta = 700
            vtc = 75
            calc = 1000
            fer = 18
        else:
            car = 4 * peso
            pro = 0.8 * peso
            gordura = peso
            vta = 700
            vtc = 75
            calc = 1000
            fer = 18
    return car, pro, gordura, vta, vtc, calc, fer

# Leitura dos dados dos alimentos
alimentos_df = pd.read_excel('/content/r.xlsx')

# Entrada dos dados do usuário
genero = input("Digite seu gênero (homem/mulher): ").upper()
peso = float(input('Digite seu peso em kilos: '))


# Cálculo dos requisitos nutricionais do usuário
car, pro, gordura, vta, vtc, calc, fer = calcular_requisitos_nutricionais(genero, peso)

# Criação do solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Definição das variáveis de decisão
quantidades_alimentos = [solver.IntVar(0, solver.infinity(), alimentos_df.iloc[i]['d']) for i in range(len(alimentos_df))]

# Restrições nutricionais
solver.Add(sum(alimentos_df.iloc[i]['c'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= car)
solver.Add(sum(alimentos_df.iloc[i]['p'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= pro)
solver.Add(sum(alimentos_df.iloc[i]['g'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= gordura)
solver.Add(sum(alimentos_df.iloc[i]['va'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= vta)
solver.Add(sum(alimentos_df.iloc[i]['vc'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= vtc)
solver.Add(sum(alimentos_df.iloc[i]['cal'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= calc)
solver.Add(sum(alimentos_df.iloc[i]['f'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= fer)

# Definição da função objetivo (minimização)
objective = solver.Objective()
for var in quantidades_alimentos:
    objective.SetCoefficient(var, 1)
objective.SetMinimization()

# Solução do problema de otimização
status = solver.Solve()

# Listas para armazenar a quantidade de nutrientes consumidos
p = []
g = []
va = []
vc = []
c = []
cal = []
f= []

if status == pywraplp.Solver.OPTIMAL:
    print("\033[32m" + 'Dieta otimizada encontrada:' + "\033[32m")
    for i in range(len(alimentos_df)):
        if quantidades_alimentos[i].solution_value() > 0:
            # Imprime os alimentos da dieta otimizada
            print("\033[93m{}\033[0m: \033[93m{}\033[0m \033[93munidades\033[0m".format(alimentos_df.iloc[i]['d'], quantidades_alimentos[i].solution_value()))
            print("\033[32m=\033[0m"*20, "\033[32mNUTRIENTES\033[0m", "\033[32m=\033[0m"*20)
            print("\033[32mProteínas:\033[0m", "\033[32m", alimentos_df.iloc[i]['p'], "\033[0m")
            print("\033[33mGorduras:\033[0m", "\033[33m", alimentos_df.iloc[i]['g'], "\033[0m")
            print("\033[34mVitamina A:\033[0m", "\033[34m", alimentos_df.iloc[i]['va'], "\033[0m")
            print("\033[35mVitamina C:\033[0m", "\033[35m", alimentos_df.iloc[i]['vc'], "\033[0m")
            print("\033[36mCarboidratos:\033[0m", "\033[36m", alimentos_df.iloc[i]['c'], "\033[0m")
            print("\033[31mCálcio:\033[0m", "\033[31m", alimentos_df.iloc[i]['cal'], "\033[0m")
            print("\033[91mFerro:\033[0m", "\033[91m", alimentos_df.iloc[i]['f'], "\033[0m")
            # Calcula e armazena a quantidade de nutrientes consumidos
            p.append(alimentos_df.iloc[i]['p'] * quantidades_alimentos[i].solution_value())
            g.append(alimentos_df.iloc[i]['g'] * quantidades_alimentos[i].solution_value())
            va.append(alimentos_df.iloc[i]['va'] * quantidades_alimentos[i].solution_value())
            vc.append(alimentos_df.iloc[i]['vc'] * quantidades_alimentos[i].solution_value())
            c.append(alimentos_df.iloc[i]['c'] * quantidades_alimentos[i].solution_value())
            cal.append(alimentos_df.iloc[i]['cal'] * quantidades_alimentos[i].solution_value())
            f.append(alimentos_df.iloc[i]['f'] * quantidades_alimentos[i].solution_value())



# Impressão dos totais de nutrientes ingeridos e recomendados
print('\033[1;32m' + f'Total de proteínas ingeridas ao dia: {sum(p)}' + '\033[0m')
print('\033[1;34m' + 'Total de proteínas necessárias ao dia: {}'.format(pro) + '\033[0m')
print('\033[1;32m' + f'Total de gorduras ingeridas ao dia: {sum(g)}' + '\033[0m')
print('\033[1;34m' + 'Total de gorduras necessárias ao dia: {}'.format(gordura) + '\033[0m')
print('\033[1;32m' + f'Total de vitamina A ingerida ao dia: {sum(va)}' + '\033[0m')
print('\033[1;34m' + 'Total de vitamina A necessária ao dia: {}'.format(vta) + '\033[0m')
print('\033[1;32m' + f'Total de vitamina C ingerida ao dia: {sum(vc)}' + '\033[0m')
print('\033[1;34m' + 'Total de vitamina C necessária ao dia: {}'.format(vtc) + '\033[0m')
print('\033[1;32m' + f'Total de carboidratos ingeridos ao dia: {sum(c)}' + '\033[0m')
print('\033[1;34m' + 'Total de carboidratos necessários ao dia: {}'.format(car) + '\033[0m')
print('\033[1;32m' + f'Total de cálcio ingerido ao dia: {sum(cal)}' + '\033[0m')
print('\033[1;34m' + 'Total de cálcio necessário ao dia: {}'.format(calc) + '\033[0m')
print('\033[1;32m' + f'Total de ferro ao dia: {sum(f)}' + '\033[0m')

print('\033[1;34m' + 'Total de ferro necessário ao dia: {}'.format(fer) + '\033[0m')

Digite seu gênero (homem/mulher): homem
Digite seu peso em kilos: 70
Dieta otimizada encontrada:
Cereais, mingau, milho, infantil: 1.0 unidades
==================== NUTRIENTES ====================
Proteínas:  6.4 
Gorduras:  1.1 
Vitamina A:  21 
Vitamina C:  109.4 
Carboidratos:  87.3 
Cálcio:  219 
Ferro:  3.0 
Leite, de vaca, integral, pó: 1.0 unidades
==================== NUTRIENTES ====================
Proteínas:  25.4 
Gorduras:  26.9 
Vitamina A:  361 
Vitamina C:  0.0 
Carboidratos:  39.2 
Cálcio:  890 
Ferro:  0.5 
Achocolatado, pó: 1.0 unidades
==================== NUTRIENTES ====================
Proteínas:  4.2 
Gorduras:  2.2 
Vitamina A:  796 
Vitamina C:  0.0 
Carboidratos:  91.2 
Cálcio:  44 
Ferro:  5.4 
Gelatina, sabores variados, pó: 1.0 unidades
==================== NUTRIENTES ====================
Proteínas:  8.9 
Gorduras:  0.0 
Vitamina A:  0 
Vitamina C:  40.0 
Carboidratos:  89.2 
Cálcio:  27 
Ferro:  0.3 
Amêndoa, torrada, salgada: 1.0 unidades
=================